---
title: Moment Problems in Probability
jupyter: julia-1.11
---



- Really cool applied math problem : can you construct a probability distribution from its moments / cumulants?
- Would such a construction be unique? What kind of associated issues can you run into with such a construction


# Naive Approach : Fourier Transform


In [ ]:
using QuadGK
using Integrals

In [ ]:
using LinearAlgebra
using Plots
using FFTW

In [ ]:
cums = [0.0, 100, 0.0, 3.0];

In [ ]:
function characteristic_function(x; cumz=cums)
    s = 0. + 0.0im
    for (n, cum) in enumerate(cumz)
        if (n < 20)
            s += cum * ((x*1.0im)^n) / factorial(n)
        else
            # Use stirling's approximation if n > 20
            s += cum * ((x*1.0im)^n) / (n^n * exp(n))
        end
    end
    return exp(s)
end

In [ ]:
characteristic_function(0.02)

In [ ]:
t = -1:0.1:1
plot(t, real.(characteristic_function.(t)), title="Real part of characteristic function", ylabel="phi(x)")

In [ ]:
y = -200:0.01:200
ϕ = characteristic_function.(y)
F = fftshift(fft(ϕ)); # Fourier transform of characteristic function gives us the distribution

In [ ]:
function P(x)
    i = argmin(abs.(y .- x))
    return abs(real(F[i]))
end

In [ ]:
#norm, _ = quadgk(P, -Inf, Inf)

In [ ]:
function prob(x)
    return P(x)
end

In [ ]:
#quadgk(prob, -Inf, Inf)

In [ ]:
plot(y, prob, title="Probability Distribution", ylabel="P(x)")

In [ ]:
import Pkg; Pkg.add("MCIntegration")
using MCIntegration

In [ ]:
x3prob(x, p) = (x^4)*prob(x)
intprob = IntegralProblem(x3prob, (-Inf, Inf))
#sol = solve(intprob, QuadGKJL(), reltol = 1e-3, abstol = 1e-3)

In [ ]:
#sol.u

In [ ]:
# Smoothing via moving averages

binsize = 150

K = abs.(real.(prob))

mov = []
xaxis = []
for i in 1:(length(enumerate(K))-binsize)
    sum = 0
    for j in 1:binsize
        sum += K[i+j-1]
    end
    push!(mov, sum/binsize)
    push!(xaxis, x[i])
end

In [ ]:
plot(xaxis, mov, title="Probability Distribution Smoothed")

Reading
1. Classical Moment Problem, Akhiezer
2. 
